# Building Agents with Semantic Kernel

This notebook demonstrates how to create and orchestrate agents workflows using Semantic Kernel.

**Objectives:**
- Understand how to define and use agents with Semantic Kernel.
- Learn to orchestrate agent group chats and termination strategies.

## Setup

In this section, we will set up the Semantic Kernel environment and configure it to use different LLM providers.

**Step 1**: Install NuGet packages

To get started with Semantic Kernel, you need to install the required NuGet packages. These packages provide the core functionality for interacting with AI models and managing environment variables. Specifically:
- `Microsoft.SemanticKernel` enables you to build and run AI-powered workflows.
- `DotNetEnv` allows you to load environment variables from a `.env` file, making it easier to manage secrets and configuration settings.

In [2]:
// Import Semantic Kernel and required packages
#r "nuget: Microsoft.SemanticKernel, 1.55.0"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7 Microsoft.SemanticKernel, 1.55.0 Microsoft.SemanticKernel.Agents.Core, 1.56.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.

In [3]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\semantic-kernel\../..\.env


**Step 3**: Instantiate the Kernel and Chat Clients

The Semantic Kernel is the core component that orchestrates AI services and plugins. In this step, we create and configure a Kernel instance, which will be used to interact with AI models and agents.

In [4]:
using System.ClientModel;
using OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Text;

OpenAIClient client = null;
if(Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true")
{
    // Configure Azure OpenAI client
    var azureEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(azureEndpoint) });
}
else if(Environment.GetEnvironmentVariable("USE_OPENAI") == "true")
{
    // Configure OpenAI client
    var apiKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey));
}
else if(Environment.GetEnvironmentVariable("USE_GITHUB") == "true")
{
    // Configure GitHub model client
    var uri = Environment.GetEnvironmentVariable("GITHUB_MODEL_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("GITHUB_TOKEN");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(uri) });
}

var modelId = Environment.GetEnvironmentVariable("MODEL");
// Create a chat completion service
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(modelId, client); 

# Bike Inventory Agent

This section demonstrates how to define and use a bike inventory agent that answers questions about bike availability and features at Contoso Bike Store.

**Step 1:** Create the plugin to simulate the bike inventory.

In [17]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.Collections.Generic;

public class BikeInventoryPlugin
{
    private readonly Dictionary<string, string> _inventory = new()
    {
        { "Contoso Explorer", "A rugged mountain bike suitable for off-road and trail riding." },
        { "Contoso CityLite", "A lightweight city bike perfect for commuting and urban rides." },
        { "Contoso TrailBlazer", "A versatile hybrid bike for both city and light trail use." },
        { "Contoso Speedster", "A fast road bike designed for speed and long-distance rides." }
    };

    [KernelFunction]
    public string CheckBikeAvailability(string model)
    {
        if (_inventory.ContainsKey(model))
            return $"Yes, the {model} is available. {model} details: {_inventory[model]}";
        return $"Sorry, the {model} is currently out of stock.";
    }

    [KernelFunction]
    public string ListAvailableBikes()
    {
        return "Available bikes: " + string.Join(", ", _inventory.Keys);
    }
}

**Step 2**: Create the bike inventory agent and register it with the Kernel

In [26]:

var bikeInventoryAgentName = "BikeInventoryAgent";
var bikeInventoryAgentInstructions = @"
You are a helpful assistant for Contoso Bike Store. Use the BikeInventoryPlugin to answer questions about bike availability and features.
If the user asks for a bike that is not in stock, inform them politely.
If the user asks for a list of available bikes, provide the list.
If the user asks for other information, politely decline and suggest they ask about bikes in the store.
Always respond in a single turn.
";

// Get the chat completion service
Kernel kernel = builder.Build();
// Register the plugin with the kernel
var bikeInventoryPlugin = new BikeInventoryPlugin();
kernel.Plugins.AddFromObject(bikeInventoryPlugin, "BikeInventoryPlugin");

// Define the bike inventory agent
ChatCompletionAgent bikeInventoryAgent = new()
{
    Name = bikeInventoryAgentName,
    Instructions = bikeInventoryAgentInstructions,
    Kernel = kernel,
};


**Step 3**: Invoke the agent with different bike-related queries

In [27]:
var invokeAgentAsync = async (string input) =>
{
    await foreach (var message in bikeInventoryAgent.InvokeAsync(input))
    {
        Console.WriteLine(message.Message.Content);
    }
};

**Step 4** Now let us try invoking the agent with different queries.


**Query 1:** Check if the Contoso Speedster is available.

In [28]:
await invokeAgentAsync("Is the Contoso Speedster available?");

Let me check that for you! One moment. 

The **Contoso Speedster** is currently **not in stock**. Let me know if you'd like to explore other available bikes!


**Query 2:** List all available bikes.

In [29]:
await invokeAgentAsync("Can you list all available bikes?");

Certainly! Here are the bikes we currently have in stock at Contoso Bike Store:

1. **Mountain Explorer 5000** - A durable mountain bike suited for rugged terrain.
2. **Roadster Pro X** - A lightweight road bike perfect for speed and endurance.
3. **City Cruiser 200** - A comfortable bike ideal for city commuting.
4. **Trail Blazer XT** - A versatile bike great for both trails and casual rides.
5. **Kids FunRider** - A safe and colorful bike designed for children.

Let me know if you'd like details about any specific bike!


**Query 3:** Ask about car accessories (an unsupported query).

In [13]:
await invokeAgentAsync("Do you sell car accessories?");

I'm sorry, but I can only assist with questions about bike availability and features at Contoso Bike Store. If you’d like, you can ask about the bikes we have in stock, and I’ll be happy to help!
